# Tests with laser HDRs

## Setup

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt 
from tqdm.auto import tqdm as progressbar
import time
%matplotlib qt5

In [ ]:
import sys
import time
sys.path.append(r"../ender")
from snapshot import MjpgSnapshotterExp


snap = MjpgSnapshotterExp("http://192.168.0.15/webcam/", ssh_str="pi@192.168.0.15")

### Test snapshotter

In [ ]:
snap.set_exposure(50) # "Ideal", i.e. red is saturated, G&B is not yet
snap.set_gain(1) # "Ideal", i.e. red is saturated, G&B is not yet
snap.send_command("-csharpness=24")
ret, img = snap.read()
plt.imshow(img)

### Get images

In [ ]:
exposures = [25, 50, 100, 500, 1000, 10000]
exposures_time = np.array(exposures) * 10e-6
images=[]
for exp in exposures:
    ret = False
    while not ret:
        print(f"Getting image for {exp=}")
        snap.set_exposure(exp)
        ret, img = snap.read()
    images.append(img)

In [ ]:
fig, axs = plt.subplots(2, len(exposures)//2)
for im, ax in zip(images, axs.flatten()):
    ax.imshow(im)

## HDR

Copiado da [documentaçõao opencv](https://docs.opencv.org/4.5.2/d2/df0/tutorial_py_hdr.html)

### Mertens
Não precisa de set-up nem dos tempos de exposição

In [ ]:
merge_mertens = cv2.createMergeMertens()
res_mertens = merge_mertens.process(images)
plt.imshow(res_mertens)

### Debevec
Precisa dos tempos de exposição. ACHO que acertei os tempos da minha camera

Update: não acertei... fica muito errado com a variável `exposure_times`. Com a variável `exposures`, que é o valor absoluto vindo da câmera, fica bem bom. Então deixa assim

In [ ]:
# Merge exposures to HDR image
merge_debevec = cv2.createMergeDebevec()
hdr_debevec = merge_debevec.process(images, times=np.array(exposures, dtype=np.float32))

tonemap1 = cv2.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())
res_debevec = np.clip(res_debevec*255, 0, 255).astype('uint8')

plt.figure()
plt.imshow(res_debevec)

### Robertson

In [ ]:
merge_robertson = cv2.createMergeRobertson()
hdr_robertson = merge_robertson.process(images, times=np.array(exposures, dtype=np.float32))

tonemap1 = cv2.createTonemap(gamma=2.2)
res_robertson = tonemap1.process(hdr_robertson)
res_robertson = np.clip(res_robertson*255, 0, 255).astype('uint8')

plt.figure()
plt.imshow(res_robertson)

### Avaliação

Avaliar usando o contraste vermelho das imagens, já que o objetivo é uma linha de laser clara

**Resultado**
* Mertens: Resultado razoável com imagens de noite e de dia. Deu imagens com o contraste vermelho bom, mas não muito consistente.
* Debevec: Funcionou *bem* com imagens de noite e de dia (superexposto de dia). Deu imagens com o contraste vermelho bem pronunciado.
* Robertson: em imagems com um componente superexposto (luz do dia), robertson deu um resultado final ruim

In [ ]:
def red_contrast(image):
	image = image.astype(np.float)
	return image[:,:, 0] - np.mean(image[:,:,1:], axis=-1)

In [ ]:
fig, axs = plt.subplots(2,3, sharex=True, sharey=True)
axs[0,0].imshow(red_contrast(res_mertens))
axs[1,0].imshow(res_mertens)
axs[0,0].set_title("Mertens")

axs[0,1].imshow(red_contrast(res_debevec))
axs[1,1].imshow(res_debevec)
axs[0,1].set_title("Debevec")

axs[0,2].imshow(red_contrast(res_robertson))
axs[1,2].imshow(res_robertson)
axs[0,2].set_title("Robertson")

### Tonemapping e CRF
usando o método Debevec, que qualitativamente deu o melhor resultado

[Código daqui](https://github.com/abhishek305/Opencv-HDR-demo/blob/master/hdr.py)

In [ ]:
# .Obtain Camera Response Function (CRF)
print("Calculating Camera Response Function (CRF) ... ")
times=np.array(exposures, dtype=np.float32)
calibrateDebevec = cv2.createCalibrateDebevec()
responseDebevec = calibrateDebevec.process(images, times)

# Merge images into an HDR linear image
print("Merging images into one HDR image ... ")
mergeDebevec = cv2.createMergeDebevec()
hdrDebevec = mergeDebevec.process(images, times, responseDebevec)

# # Tonemap using Drago's method to obtain 24-bit color image
print("Tonemaping using Drago's method ... ")
tonemapDrago = cv2.createTonemapDrago(2.2, 0.7)
ldrDrago = tonemapDrago.process(hdrDebevec)
ldrDrago = 3 * ldrDrago
res_drago = np.clip(ldrDrago*255, 0, 255).astype('uint8')


# # Tonemap using Reinhard's method to obtain 24-bit color image
print("Tonemaping using Reinhard's method ... ")
tonemapReinhard = cv2.createTonemapReinhard(2.2, 0,0,0)
ldrReinhard = tonemapReinhard.process(hdrDebevec)
res_Reinhard = np.clip(ldrReinhard*255, 0, 255).astype('uint8')

# # Tonemap using Mantiuk's method to obtain 24-bit color image
print("Tonemaping using Mantiuk's method ... ")
tonemapMantiuk = cv2.createTonemapMantiuk(2.2,0.5, 1.2)
ldrMantiuk = tonemapMantiuk.process(hdrDebevec)
ldrMantiuk = 3 * ldrMantiuk
res_Mantiuk = np.clip(ldrMantiuk*255, 0, 255).astype('uint8')


In [ ]:
fig, axs = plt.subplots(2,4, sharex=True, sharey=True)

plt.title("Tonemaps for Debevec Result")

axs[0, 0].imshow(res_debevec)
axs[1, 0].imshow(red_contrast(res_debevec))
axs[0, 0].set_title("Gamma Tonemap")

axs[0, 1].imshow(res_drago)
axs[1, 1].imshow(red_contrast(res_drago))
axs[0, 1].set_title("Drago Tonemap")

axs[0, 2].imshow(res_Reinhard)
axs[1, 2].imshow(red_contrast(res_Reinhard))
axs[0, 2].set_title("Reinhard Tonemap")

axs[0, 3].imshow(res_Mantiuk)
axs[1, 3].imshow(red_contrast(res_Mantiuk))
axs[0, 3].set_title("Mantiuk Tonemap")

## Resultado

Melhor overall é o debevec, menos parâmetros (só o gamma)

In [ ]:
cv2.imwrite("images/debevec_board.png", cv2.cvtColor(res_debevec, cv2.COLOR_RGB2BGR))